# Práctica 6 - Support Vector Machines {-}
Álvar Domingo Fernández y Pablo Jurado López

### Preparación inicial {-}
A continuación se importan todas las librerías y funciones externas que serán utilizadas en esta práctica.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy.optimize import minimize
from sklearn.svm import SVC
from process_email import email2TokenList
import codecs
from get_vocab_dict import getVocabDict
import glob
import sklearn.model_selection as ms
from sklearn.metrics import confusion_matrix
import time

## 1.1- Kernel lineal {-}
Hemos clasificado los datos contenidos en ex6data1.mat con un SVM con un parámetro de regularización C=1 y una función de kernel lineal, y lo hemos renderizado en una gráfica. Posteriormente hemos repetido el mismo proceso, esta vez con un parámetro de regularización C = 100.

In [ ]:
def apartado1_1():
    data = loadmat('ex6data1.mat')
    X, y = data['X'], data['y'].ravel() 
    svm = SVC(kernel= 'linear', C = 1)
    visualize_boundary(X, y, svm, "apartado1_1_C1")
    svm = SVC(kernel= 'linear', C = 100)
    visualize_boundary(X, y, svm, "apartado1_1_C100")

![SVM lineal con C = 1](apartado1_1_C1.png)

![SVM lineal con C = 100](apartado1_1_C100.png)

## 1.2- Kernel gaussiano {-}
Hemos repetido el proceso del apartado anterior, pero esta vez hemos utilizado una función de kernel gaussiana

In [ ]:
def apartado1_2():
    data = loadmat('ex6data2.mat')
    X, y = data['X'], data['y'].ravel() 
    C = 1
    sigma = 0.1 
    svm = SVC(kernel= 'rbf', C = C, gamma=1 / (2 * sigma**2))
    visualize_boundary(X, y, svm, "apartado1_2")

![SVM con kernel gaussiano](apartado1_2.png)

## 1.3- Elección de los parámetros C y $\sigma$ {-}
Hemos utilizado la función svm.score y probado con los siguientes valores para C y $\sigma$: 0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30.

In [ ]:
def apartado1_3():
    data = loadmat('ex6data3.mat')
    X = data['X']
    y = data['y'].ravel()
    Xval = data['Xval']
    yval = data['yval'].ravel()
    values = [0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30]
    n = len(values)
    scores = np.zeros((n, n))

    for i in range(n):
        C = values[i]
        for j in range(n):
            sigma = values[j]
            svm = SVC(kernel='rbf', C = C, gamma= 1 / (2 * sigma **2))
            svm.fit(X, y.ravel())
            scores[i, j] = svm.score(Xval, yval)

    print("Error mínimo: {}".format(1 - scores.max())) 
    C_opt = values[scores.argmax()//n]
    sigma_opt = values[scores.argmax()%n]
    print("C óptimo: {}, sigma óptimo: {}".format(C_opt, sigma_opt))

    svm = SVC(kernel= 'rbf', C = C_opt, gamma= 1 / (2 * sigma_opt ** 2))
    visualize_boundary(X, y, svm, "apartado1_3")

El resultado obtenido es el siguiente:
Error mínimo: 0.03500000000000003
C óptimo: 1, sigma óptimo: 0.1